# Transforming Code into Beautiful, Idiomatic Python (by Raymond Hettinger)

When you see this, do that instead:
    
* Replace traditional index manipulation with Python's core looping idioms.
* Learn advanced techniques with for-else clauses and the two argument form of iter().
* Improve your craftmanship and aim for clean, fast, idiomatic Python code.

In [127]:
import  functools
from collections import defaultdict
import argparse
import os
from collections import ChainMap,deque
import urllib
from cachetools import cached, TTLCache
import _thread

## Looping over a range of numbers

In [2]:
for i in [0,1,2,3,4,5]:
    print(i**2)

0
1
4
9
16
25


In [3]:
for i in range(6):
    print(i**2)

0
1
4
9
16
25


## Looping over a collection

In [5]:
colors = ['red','green','blue','yellow']

In [6]:
for i in range(len(colors)):
    print(colors[i])

red
green
blue
yellow


In [7]:
for color in colors:
    print(color)

red
green
blue
yellow


## Looping backwards

In [10]:
for i in range(len(colors)-1,-1,-1):
    print(colors[i])

yellow
blue
green
red


In [9]:
for color in reversed(colors):
    print(color)

yellow
blue
green
red


## Looping over a collection and indices

In [11]:
for i in range(len(colors)):
    print(i,'->',colors[i])

0 -> red
1 -> green
2 -> blue
3 -> yellow


In [12]:
for i,color in enumerate(colors):
    print(i,'->',color)

0 -> red
1 -> green
2 -> blue
3 -> yellow


## Looping over two collections

In [13]:
names = ['raymond','rachel','mathew']

In [14]:
n = min(len(names),len(colors))
for i in range(n):
    print(names[i],'->',colors[i])

raymond -> red
rachel -> green
mathew -> blue


In [15]:
for name,color in zip(names,colors):
    print(name,'->',color)

raymond -> red
rachel -> green
mathew -> blue


## Looping in sorted order

In [16]:
for color in sorted(colors):
    print(color)

blue
green
red
yellow


In [17]:
for color in sorted(colors, reverse=True):
    print(color)

yellow
red
green
blue


## Custom sort order

In [18]:
def compare_length(c1,c2):
    if(len(c1)<len(c2)):
        return -1
    if(len(c1)>len(c2)):
        return 1
    return 0

In [23]:
print(sorted(colors, key=functools.cmp_to_key(compare_length)))

['red', 'blue', 'green', 'yellow']


In [25]:
print(sorted(colors, key=len))

['red', 'blue', 'green', 'yellow']


## Call a function until a sentinel value

In [26]:
blocks = []
while True:
    block = f.read
    if block == '':
        break
    blocks.append(block)
    
blocks = []
for block in iter(partial(f.read,32),''):
    blocks.append(block)

NameError: name 'f' is not defined

## Distinguishing multiple exit points in loops

In [28]:
# Flag slows down your code and make it less readable
def find(seq, target):
    found = False
    for i, value in enumerate(seq):
        if value==target:
            found=True
            break
        if not found:
            return -1
        return i

In [ ]:
def find(seq, target):
    for i,value in enumerate(seq):
        if value==target:
            break
    else:
        return -1
    return i

## Dictionary Skills

* Mastering dictionaries is a fundamental Python skill.
* They are fundamental tool for expressing relationships, linking, counting and grouping.

In [30]:
d = {'mathew':'blue','rachel':'green','raymond':'red'}

## Looping over dictionary keys

In [32]:
for k in d:
    print(k)

mathew
rachel
raymond


In [33]:
for k in d.keys():
    if k.startswith('r'):
        del d[k]

RuntimeError: dictionary changed size during iteration

In [34]:
# Erase every element (key,value) that has a key that starts with 'r'
d = {k:d[k] for k in d if not k.startswith('r')}

In [37]:
for k in d:
    print(k)

mathew


## Looping over a dictionary keys and values

In [39]:
d = {'mathew':'blue','rachel':'green','raymond':'red'}
for k in d:
    print(k,'->',d[k])

mathew -> blue
rachel -> green
raymond -> red


In [40]:
# Use of tupple unpacking
for k,v in d.items():
    print(k,'->',v)

mathew -> blue
rachel -> green
raymond -> red


## Construct a dictionary from pairs

In [42]:
names = ['raymond','rachel','mathew']
colors = ['red','green','blue','yellow']

In [43]:
d = dict(zip(names,colors))

In [45]:
for k,v in d.items():
    print(k,'->',v)

raymond -> red
rachel -> green
mathew -> blue


In [47]:
d = dict(enumerate(names))
d

{0: 'raymond', 1: 'rachel', 2: 'mathew'}

## Counting with dictionaries

In [48]:
colors = ['red','green','red','blue','green','red']

In [51]:
d = {}
for color in colors:
    if color not in d:
        d[color] = 0
    d[color] +=1
    
d

{'red': 3, 'green': 2, 'blue': 1}

In [53]:
d = {}
for color in colors:
    d[color] = d.get(color,0) + 1
    
d

{'red': 3, 'green': 2, 'blue': 1}

In [57]:
d = defaultdict(int)
for color in colors:
    d[color] +=1
    
d

defaultdict(int, {'red': 3, 'green': 2, 'blue': 1})

## Grouping with dictionaries -- Part I

In [58]:
names = ['raymond','rachel','mathew','roger','betty','melissa','judith','charlie']

In [60]:
d = {}
for name in names:
    key = len(name)
    if key not in d:
        d[key] = []
    d[key].append(name)
    
d

{7: ['raymond', 'melissa', 'charlie'],
 6: ['rachel', 'mathew', 'judith'],
 5: ['roger', 'betty']}

## Grouping with dictionaries -- Part II

In [61]:
d = {}
for name in names:
    key = len(name)
    d.setdefault(key,[]).append(name)
    
d

{7: ['raymond', 'melissa', 'charlie'],
 6: ['rachel', 'mathew', 'judith'],
 5: ['roger', 'betty']}

In [62]:
d = defaultdict(list)
for name in names:
    key = len(name)
    d[key].append(name)
    
d

defaultdict(list,
            {7: ['raymond', 'melissa', 'charlie'],
             6: ['rachel', 'mathew', 'judith'],
             5: ['roger', 'betty']})

## Is a dictionary popitem() atomic?

In [63]:
d = {'mathew':'blue', 'rachel':'green', 'raymond':'red'}

In [64]:
while d:
    key,value = d.popitem()
    print(key,'->',value)

raymond -> red
rachel -> green
mathew -> blue


## Linking dictionaries

In [86]:
defaults = {'color':'red','user':'guest'}
parser = argparse.ArgumentParser()
parser.add_argument('-u','--user')
parser.add_argument('-c','--color')
namespace = parser.parse_args([])
command_line_args = {k:v for k,v in vars(namespace).items() if v}

In [87]:
d = defaults.copy()
d.update(os.environ)
d.update(command_line_args)

In [88]:
d_chainmap = ChainMap(command_line_args,os.environ,defaults)

## Improving Clarity

* Positional arguments and indicies are nice.
* Keywords and names are better.
* The first way in inconvenient for the computer
* The second corresponds to how human's think.

## Clarity function calls with keyword arguments

In [93]:
# Hours of programmer time vs micro-seconds on computer #
#=======================================================#

#twitter_search('@obama',False,20,True)
#twitter_search('@obama',retweets=False,numtweets=20,popular=True)

## Clarify multiple return values with named tuples

In [95]:
# doctest.testmod()
# (0,4)

# doctest.testmod()
# TestResults(failed=0, attempted=4)

#TestResults = namedtuple('TestResults',['failed','attempted'])

## Unpacking sequences

In [96]:
p = 'Raymond','Hettinger', 0x30,'python@example.com'
fname = p[0]
lname = p[1]
age = p[2]
email = p[3]

fname, lname, age, email = p

## Updating multiple state variables

In [97]:
def fibonacci(n):
    x=0
    y=1
    for i in range(n):
        print(x)
        t = y
        y = x+y
        x = t

In [98]:
def fibonacci2(n):
    x,y=0,1
    for i in range(n):
        print(x)
        x,y = y,x+y

## Efficiency 

* An optimization fundamental rule
* Don't cause data to move around unnecessarily
* It takes only a little care to avoid O(n**2) behavior instead of linear behavior.

## Concatenating strings

In [99]:
names = ['raymond','rachel','mathew','roger','betty','melissa','judith','charlie']

In [100]:
s = names[0]
for name in names:
    s+=', ' + name
print(s)

raymond, raymond, rachel, mathew, roger, betty, melissa, judith, charlie


In [102]:
print(', '.join(names))

raymond, rachel, mathew, roger, betty, melissa, judith, charlie


## Updating sequences 

In [103]:
del names[0]
names.pop(0)
names.insert(0,'mark')

In [104]:
names

['mark', 'mathew', 'roger', 'betty', 'melissa', 'judith', 'charlie']

In [107]:
names = deque(['raymond','rachel','mathew','roger','betty','melissa','judith','charlie'])

In [108]:
del names[0]
names.popleft()
names.appendleft('mark')

## Decorators and Context Managers 

* Helps separate business logic from administrative logic.
* Clean, beautiful tools for factoring code and improving code reuse.
* Good naming is essential
* Remember the Spiderman rule: With great power, comes great responsability!

## Using decorators to factor-out administrative logic

In [112]:
def web_lookup(url, saved={}):
    if url in saved:
        return saved[url]
    page = urllib.urlopen(url).read()
    saved[url] = page
    return page

In [119]:
cache = TTLCache(maxsize=100, ttl=300)

@cached(cache)
def web_lookup2(url):
    return urllib.urlopen(url).read()

## Caching a decorator 

In [120]:
def cache(func):
    saved = {}
    wraps(func)
    def newfunc(*args):
        if args in saved:
            return newfunc(*args)
        result = func(*args)
        saved[args] = result
        return result
    return newfunc

## Factor out temporary contexts

In [122]:
#old_context = getcontext().copy()
#getcontext().prec = 50
#print(Decimal(355)/Decimal(113))
#setcontext(old_context)

In [123]:
#with localcontext(Context(prec=50)):
#    print(Decimal(355)/Decimal(113))

## How to open and close files

In [124]:
#f = open('data.txt')
#try:
#    data = f.read()
#finally:
#    f.close()

In [125]:
# with open('data.txt') as f:
#data = f.read()

## How to use locks

In [129]:
# Make a lock
#lock = threading.Lock()

# Old-way to use a lock
#lock.acquire()
#try:
#    print('Critical section 1')
#    print('Critical section 2')
#finally:
#    lock.release()
    
# New way to use a lock
#with lock:
#    print('Critical section 1')
#    print('Critical section 2')

## Factor out temporary contexts

In [130]:
#try:
#    os.remove('somefile.tmp')
#except OSError:
#    pass

#with ignored(OSError):
#    os.remove('somefile.tmp')

In [131]:
#with open('help.txt','w') as f:
#    oldstdout = sys.stdout
#    sys.stdout = f
#    try:
#        help(pow)
#    finally:
#        sys.stdout = oldstdout

## Concise Expressive One-Liners

Two conflicting rules:
* Don't put too much on one line
* Don't break atoms of thought into subatomic particles

Raymond's rule:
* One logical line of code equals one sentence in English

## List Comprehensions and Generator Expressions

In [132]:
result = []
for i in range(10):
    s = i**2
    result.append(s)
print(sum(result))

285


In [134]:
print(sum([i**2 for i in range(10)]))

285


In [135]:
print(sum(i**2 for i in range(10)))

285
